In [36]:
using LowLevelFEM
gmsh.initialize()

In [37]:
gmsh.open("truss.geo")

Info    : Reading 'truss.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 60%] Meshing curve 2 (Line)
Info    : Done meshing 1D (Wall 0.000165588s, CPU 0.000164s)
Info    : 3 nodes 5 elements
Info    : Done reading 'truss.geo'


In [38]:
mat = material("rod")
prob = Problem([mat], type=:Truss)

Problem("truss", :Truss, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("rod", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 3)

In [39]:
K = stiffnessMatrix(prob)

9×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 19 stored entries:
  1000.0   ⋅    ⋅   -1000.0        ⋅      ⋅       ⋅         ⋅      ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 
 -1000.0   ⋅    ⋅    1715.54    357.771   ⋅   -715.542  -357.771   ⋅ 
      ⋅    ⋅    ⋅     357.771   178.885   ⋅   -357.771  -178.885   ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 
      ⋅    ⋅    ⋅    -715.542  -357.771   ⋅    715.542   357.771   ⋅ 
      ⋅    ⋅    ⋅    -357.771  -178.885   ⋅    357.771   178.885   ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 

In [40]:
supp0 = displacementConstraint("rod", uz=0)
suppA = displacementConstraint("A", ux=0, uy=0)
suppB = displacementConstraint("B", ux=0, uy=0)
force = load("C", fy=-1)

("C", 0, -1, 0)

In [41]:
f = loadVector(prob, [force])

VectorField

9×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0
 -1.0
  0.0
  0.0
  0.0
  0.0

In [42]:
applyBoundaryConditions!(K, f, [supp0, suppA, suppB])

In [43]:
K

9×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 11 stored entries:
 1.0   ⋅    ⋅       ⋅        ⋅      ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅       ⋅        ⋅      ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0      ⋅        ⋅      ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1715.54   357.771   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    357.771  178.885   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅       ⋅        ⋅     1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅       ⋅        ⋅      ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅       ⋅        ⋅      ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅       ⋅        ⋅      ⋅    ⋅    ⋅   1.0

In [44]:
q = K \ f

VectorField

9×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0020000000000000005
 -0.009590169943749475
  0.0
  0.0
  0.0
  0.0

In [45]:
q.a

9×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0020000000000000005
 -0.009590169943749475
  0.0
  0.0
  0.0
  0.0

In [46]:
q2 = solveDisplacement(prob, [force], [supp0, suppA, suppB])

VectorField

9×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0020000000000000005
 -0.009590169943749475
  0.0
  0.0
  0.0
  0.0

In [47]:
showDoFResults(q, :uvec)

0

In [48]:
showDoFResults(q2, :uvec)

1

In [49]:
N = solveAxialForce(q)

ScalarField

2-element Vector{Matrix{Float64}}:
 [-2.2360679774997894; -2.2360679774997894;;]
 [2.0000000000000004; 2.0000000000000004;;]

In [50]:
showElementResults(N, :scalar)

2

In [51]:
showElementResults(N, :scalar, smooth=true)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [52]:
showDoFResults(N, :scalar)

4

In [53]:
N.A

2-element Vector{Matrix{Float64}}:
 [-2.2360679774997894; -2.2360679774997894;;]
 [2.0000000000000004; 2.0000000000000004;;]

In [54]:
N.numElem

2-element Vector{Int64}:
 4
 5

In [55]:
N1 = elementsToNodes(N)

ScalarField

3×1 Matrix{Float64}:
  2.0000000000000004
 -0.11803398874989446
 -2.2360679774997894

In [56]:
N1.a

3×1 Matrix{Float64}:
  2.0000000000000004
 -0.11803398874989446
 -2.2360679774997894

In [57]:
showDoFResults(N1, :scalar)

5

In [58]:
K = stiffnessMatrix(prob)

9×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 19 stored entries:
  1000.0   ⋅    ⋅   -1000.0        ⋅      ⋅       ⋅         ⋅      ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 
 -1000.0   ⋅    ⋅    1715.54    357.771   ⋅   -715.542  -357.771   ⋅ 
      ⋅    ⋅    ⋅     357.771   178.885   ⋅   -357.771  -178.885   ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 
      ⋅    ⋅    ⋅    -715.542  -357.771   ⋅    715.542   357.771   ⋅ 
      ⋅    ⋅    ⋅    -357.771  -178.885   ⋅    357.771   178.885   ⋅ 
      ⋅    ⋅    ⋅        ⋅         ⋅      ⋅       ⋅         ⋅      ⋅ 

In [59]:
M = massMatrix(prob)

9×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 5 stored entries:
 7.85e-7   ⋅    ⋅    ⋅           ⋅           ⋅    ⋅           ⋅           ⋅ 
  ⋅        ⋅    ⋅    ⋅           ⋅           ⋅    ⋅           ⋅           ⋅ 
  ⋅        ⋅    ⋅    ⋅           ⋅           ⋅    ⋅           ⋅           ⋅ 
  ⋅        ⋅    ⋅   1.83819e-6   ⋅           ⋅    ⋅           ⋅           ⋅ 
  ⋅        ⋅    ⋅    ⋅          5.26594e-7   ⋅    ⋅           ⋅           ⋅ 
  ⋅        ⋅    ⋅    ⋅           ⋅           ⋅    ⋅           ⋅           ⋅ 
  ⋅        ⋅    ⋅    ⋅           ⋅           ⋅   1.05319e-6   ⋅           ⋅ 
  ⋅        ⋅    ⋅    ⋅           ⋅           ⋅    ⋅          5.26594e-7   ⋅ 
  ⋅        ⋅    ⋅    ⋅           ⋅           ⋅    ⋅           ⋅           ⋅ 

In [60]:
f = loadVector(prob, [force])

VectorField

9×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0
 -1.0
  0.0
  0.0
  0.0
  0.0

In [61]:
applyBoundaryConditions!(K, M, f, [supp0, suppA, suppB])

In [62]:
dt = largestPeriodTime(K, M)

0.00048604587786191223

In [63]:
u0 = initialDisplacement(prob, "rod")

VectorField

9×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [64]:
v0 = initialVelocity(prob, "rod")

VectorField

9×1 Matrix{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [65]:
u, v = HHT(K, M, f, u0, v0, 5dt, dt / 10)

VectorField

9×50 Matrix{Float64}:
 0.0   0.0           0.0           0.0         …   0.0          0.0
 0.0   0.0           0.0           0.0             0.0          0.0
 0.0   0.0           0.0           0.0             0.0          0.0
 0.0   0.000141963   0.000860341   0.00238466      0.00310784   0.00174763
 0.0  -0.00191572   -0.0066699    -0.0121219      -0.0152395   -0.00997707
 0.0   0.0           0.0           0.0         …   0.0          0.0
 0.0   0.0           0.0           0.0             0.0          0.0
 0.0   0.0           0.0           0.0             0.0          0.0
 0.0   0.0           0.0           0.0             0.0          0.0

VectorField

9×50 Matrix{Float64}:
 0.0    0.0         0.0        0.0     …    0.0       0.0       0.0
 0.0    0.0         0.0        0.0          0.0       0.0       0.0
 0.0    0.0         0.0        0.0          0.0       0.0       0.0
 0.0    5.84153    23.7186    39.0046     -20.4516  -32.5597  -23.4109
 0.0  -78.8286   -116.799   -107.541       30.6957   90.4171  126.125
 0.0    0.0         0.0        0.0     …    0.0       0.0       0.0
 0.0    0.0         0.0        0.0          0.0       0.0       0.0
 0.0    0.0         0.0        0.0          0.0       0.0       0.0
 0.0    0.0         0.0        0.0          0.0       0.0       0.0

(, )

In [66]:
showDoFResults(u, :uvec)

6

In [67]:
mod = solveModalAnalysis(prob, constraints=[supp0, suppA, suppB], n=2, directSolver=true)

LowLevelFEM.Eigen([2057.4181095857193, 5292.639058387366], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], Problem("truss", :Truss, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("rod", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0)], 1.0, 3))

In [69]:
showModalResults(mod)

7

In [70]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [71]:
gmsh.finalize()